# Benchmarking performance

## Import the required libraries


In [1]:
import timeit

import numpy as np

import pyvinecopulib as pv

from collections import defaultdict


def print_results(results, name):
  print(
    f"{name}: min={min(results) * 1000:.6f}, mean={np.mean(results) * 1000:.6f}, max={max(results) * 1000:.6f}"
  )

## Set the parameters for the benchmark

In [2]:
# TODO: Check TLL because compiler might have large impact on performance there

n = 1000
d = 5
repeats = 5


def generate_data(n, d, seed=0):
  np.random.seed(seed)
  # Generate data
  x = np.random.normal(size=n)[:, np.newaxis] * np.ones(
    (n, d)
  ) + 0.5 * np.random.normal(size=(n, d))

  # Convert data to pseudo-observations
  u = pv.to_pseudo_obs(x)

  return u


def generate_data_bicop(n, seed):
  cop = pv.Bicop(family=pv.gaussian, parameters=np.array([[0.5]]))
  return cop.simulate(n, seeds=[seed])

## Bivariate copula benchmark

In [3]:
def itau_bicop(u):
  pv.Bicop.from_data(
    u, controls=pv.FitControlsBicop(family_set=pv.itau, num_threads=1)
  )


def itau_par_method_bicop(u):
  pv.Bicop.from_data(
    u,
    controls=pv.FitControlsBicop(
      family_set=pv.itau, parametric_method="itau", num_threads=1
    ),
  )


def tll_bicop(u):
  pv.Bicop.from_data(
    u, controls=pv.FitControlsBicop(family_set=[pv.tll], num_threads=1)
  )


functions_to_benchmark_bicop = {
  "itau": itau_bicop,
  "itau_par_method": itau_par_method_bicop,
  "tll": tll_bicop,
}

# Benchmark the functions
results_bicop = defaultdict(list)
for seed in range(repeats):
  u = generate_data_bicop(n, seed)
  for name, function in functions_to_benchmark_bicop.items():
    tic = timeit.default_timer()
    function(u)
    toc = timeit.default_timer()
    results_bicop[name].append(toc - tic)


# Display benchmark results
print("Unit (milliseconds, bicop):")
for name, result in results_bicop.items():
  print_results(result, name)

Unit (milliseconds, bicop):
itau: min=69.619952, mean=108.542584, max=138.082729
itau_par_method: min=10.003054, mean=10.805146, max=12.116266
tll: min=5.403543, mean=5.558469, max=5.966208


## Vine copula benchmark

In [4]:
# Various configurations of the `vinecop` function
controls_itau = pv.FitControlsVinecop(family_set=pv.itau, num_threads=1)
controls_itau_par_method = pv.FitControlsVinecop(
  family_set=pv.itau, parametric_method="itau", num_threads=1
)
controls_tll = pv.FitControlsVinecop(family_set=[pv.tll], num_threads=1)


# Define two different configurations of the `vinecop` function
def vinecop_itau(u):
  return pv.Vinecop.from_data(u, controls=controls_itau)


def vinecop_itau_par_method(u):
  return pv.Vinecop.from_data(u, controls=controls_itau_par_method)


def vinecop_tll(u):
  return pv.Vinecop.from_data(u, controls=controls_tll)


functions_to_benchmark = {
  "itau": vinecop_itau,
  "itau_par_method": vinecop_itau_par_method,
  "tll": vinecop_tll,
}

# Benchmark the functions
results = defaultdict(list)
for seed in range(repeats):
  u = generate_data(n, d, seed)
  for name, function in functions_to_benchmark.items():
    tic = timeit.default_timer()
    function(u)
    toc = timeit.default_timer()
    results[name].append(toc - tic)


# Display benchmark results
print("Unit (milliseconds):")
for name, result in results.items():
  print_results(result, name)

# # Benchmark in C++ directly
# results_cpp = defaultdict(list)
# for seed in range(repeats):
#   u = generate_data(n, d, seed)
#   results_cpp_tmp = pv.pyvinecopulib_ext.benchmark(u)
#   for key, value in zip(functions_to_benchmark.keys(), results_cpp_tmp):
#     results_cpp[key].append(value)

# # Display benchmark results
# print("Unit (milliseconds, C++):")
# for name, result in results_cpp.items():
#   print_results(result, name)

Unit (milliseconds):
itau: min=1051.921016, mean=1229.718778, max=1457.615717
itau_par_method: min=131.477031, mean=138.356589, max=151.956707
tll: min=85.055594, mean=90.453178, max=96.229217
